# `transformers` meets `bitsandbytes` for democratzing Large Language Models (LLMs) through 4bit quantization

<center>
<img src="https://github.com/huggingface/blog/blob/main/assets/96_hf_bitsandbytes_integration/Thumbnail_blue.png?raw=true" alt="drawing" width="700" class="center"/>
</center>

Welcome to this notebook that goes through the recent `bitsandbytes` integration that includes the work from XXX that introduces no performance degradation 4bit quantization techniques, for democratizing LLMs inference and training.

In this notebook, we will learn together how to load a large model in 4bit (`gpt-neo-x-20b`) and train it using Google Colab and PEFT library from Hugging Face 🤗.

[In the general usage notebook](https://colab.research.google.com/drive/1ge2F1QSK8Q7h0hn3YKuBCOAS0bK8E0wf?usp=sharing), you can learn how to propely load a model in 4bit with all its variants.

If you liked the previous work for integrating [*LLM.int8*](https://arxiv.org/abs/2208.07339), you can have a look at the [introduction blogpost](https://huggingface.co/blog/hf-bitsandbytes-integration) to lean more about that quantization method.


In [1]:
import numpy as np
from scipy.special import softmax
import pdb
import pandas as pd
import math
from typing import List
import random
import argparse
import torch


def sent_scoring(model_tokenizer, text, cuda, score_type="loss", output_attentions=False, length_normalize=False):
    model = model_tokenizer[0]
    tokenizer = model_tokenizer[1]
    assert model is not None
    assert tokenizer is not None
    encoded_text = tokenizer.encode(text)
    input_ids = torch.tensor(encoded_text).unsqueeze(0)
    if cuda:
        input_ids = input_ids.to('cuda')
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids, output_attentions=output_attentions)
    loss, logits = outputs[:2]

    sentence_prob = loss.item()
    if score_type == "prob":
        if length_normalize:
            mult = 2
        else:
            mult = len(encoded_text)

        sentence_prob = math.exp(-1.0 * loss * (mult - 1))

    if output_attentions:
        attn = outputs["attentions"]
        return sentence_prob, attn, input_ids

    return sentence_prob

def confusion_matrix(P_forward_1, P_forward_2, P_backward_1, P_backward_2):
    correct_forward = len(np.where(np.array(P_forward_1) >= 0.5)[0]) + len(np.where(np.array(P_forward_2) >=0.5)[0])
    wrong_forward = len(P_forward_1) + len(P_forward_2) - correct_forward

    correct_backward = len(np.where(np.array(P_backward_1) >= 0.5)[0]) + len(np.where(np.array(P_backward_2) >=0.5)[0])
    wrong_backward = len(P_backward_1) + len(P_backward_2) - correct_backward

    print("correct forward", correct_forward, "wrong forward", wrong_forward, "correct backward", correct_backward, "wrong_backward", wrong_backward)

    results = {
        "correct_forward": correct_forward,
        "wrong_forward": wrong_forward,
        "correct_backward": correct_backward,
        "wrong_backward": wrong_backward
    }

    return results

from tqdm import tqdm

def evaluate_model(model, tokenizer, test_set, middle_phrase="", use_prefix=0, verbose=True, score_type="prob", use_cuda=False, return_acc=False, total = 1094) -> tuple:
    preds = []
    labels = []
    x_1 = []
    x_2 = []
    y_1 = []
    y_2 = []
    P_x_1 = []
    P_x_2 = []
    P_y_1 = []
    P_y_2 = []
    P_x_1_y_1 = []
    P_x_1_y_2 = []
    P_x_2_y_1 = []
    P_x_2_y_2 = []
    P_x_1_correct = []
    P_x_2_correct = []
    P_y_1_correct = []
    P_y_2_correct = []
    correct = 0

    for i, metaphor_data in tqdm(enumerate(test_set), total = total):
        ctx, p1, p2 = metaphor_data["startphrase"], metaphor_data["ending1"], metaphor_data["ending2"]
        labels.append(int(metaphor_data["labels"]))
        if use_prefix > 0:
            prefix_prompt = select_prefix_prompts(prompt_file, use_prefix) if use_prefix else ""
        else:
            prefix_prompt = ""

        sent1 = prefix_prompt + ctx + ". " + middle_phrase + p1 + "."
        sent2 = prefix_prompt + ctx + ". " + middle_phrase + p2 + "."

        score1 = sent_scoring((model, tokenizer), sent1, use_cuda, score_type=score_type)
        score2 = sent_scoring((model, tokenizer), sent2, use_cuda, score_type=score_type)

        if score_type == "loss":
            pred = 0 if score1 < score2 else 1
        else:
            pred = 1 if score1 < score2 else 0

        pred_sent = sent1 if pred == 0 else sent2

        if i % 2 == 0:
            x_1.append(ctx)
            x_1_score = sent_scoring((model, tokenizer), ctx + ".", use_cuda, score_type=score_type)
            P_x_1.append(x_1_score)
            y_1.append(p1)
            y_2.append(p2)
            y1_score = sent_scoring((model, tokenizer), p1 + ".", use_cuda, score_type=score_type)
            y2_score = sent_scoring((model, tokenizer), p2 + ".", use_cuda, score_type=score_type)
            P_y_1.append(y1_score)
            P_y_2.append(y2_score)

            P_x_1_y_1.append(score1)
            P_x_1_y_2.append(score2)
            P_x_1_correct.append(score1/(score1 + score2))

        else:
            x_2.append(ctx)
            x_2_score = sent_scoring((model, tokenizer), ctx + ".", use_cuda, score_type=score_type)
            P_x_2.append(x_2_score)
            P_x_2_y_1.append(score1)
            P_x_2_y_2.append(score2)
            P_x_2_correct.append(score2/(score1 + score2))

            P_y_1_correct.append(P_x_1_y_1[-1]/(P_x_1_y_1[-1] + score1))
            P_y_2_correct.append(score2/(P_x_1_y_2[-1] + score2))

        if verbose:
            print(f"Q: {ctx}: 1. {p1} 2. {p2}")
            print(f"model says '{pred_sent}' is more likely")
            print("\n")
        if pred == metaphor_data["labels"]:
            correct += 1
        preds.append(pred)

    cols = {"x_1": x_1, "x_2": x_2, "y_1": y_1, "y_2": y_2, "P(x_1)": P_x_1, "P(x_2)": P_x_2, "P(y_1)": P_y_1, "P(y_2)": P_y_2,
        "P(x_1, y_1)": P_x_1_y_1, "P(x_1, y_2)": P_x_1_y_2, "P(x_2, y_1)": P_x_2_y_1, "P(x_2, y_2)": P_x_2_y_2,
        "P(y_1|x_1)": P_x_1_correct, "P(y_2|x_2)": P_x_2_correct, "P(x_1|y_1)": P_y_1_correct, "P(x_2|y_2)": P_y_2_correct}
    out_df = pd.DataFrame(cols)

    if return_acc:
        return correct/len(preds), out_df, preds, labels

    return out_df, preds, labels

def compute_stats(total_df: pd.DataFrame, all_preds: List, all_labels: List) -> None:
    print("overall accuracy: ")
    accuracyy = len(np.where(np.array(all_preds) == np.array(all_labels))[0])/len(all_labels)
    print(accuracyy)
    print("confusion matrix: ")
    matrix_dic = confusion_matrix(list(total_df["P(y_1|x_1)"]), list(total_df["P(y_2|x_2)"]), list(total_df["P(x_1|y_1)"]), list(total_df["P(x_2|y_2)"]))

    return accuracyy, matrix_dic


In [ ]:
!pip uninstall datasets -y
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 11.6 MB/s eta 0:00:00


In [2]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.0 MB/s eta 0:00:00


In [3]:
!pip install -q -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.6 MB/s eta 0:00:00


In [4]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00


In [5]:
from datasets import load_dataset

First let's load the model we are going to use - GPT-neo-x-20B! Note that the model itself is around 40GB in half precision

In [6]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.4 MB/s eta 0:00:00


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "microsoft/phi-1_5"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The repository for microsoft/phi-1_5 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/phi-1_5.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- configuration_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


The repository for microsoft/phi-1_5 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/phi-1_5.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- modeling_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

## Metaphor Probabilities (zero-shot) for the pretrained model

In [8]:
dataset = load_dataset("nightingal3/fig-qa")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [9]:
dataset['validation']

Dataset({
    features: ['startphrase', 'ending1', 'ending2', 'labels', 'valid'],
    num_rows: 1094
})

In [10]:
subset_test_dataset = dataset['validation'].select(range(500))

In [ ]:
out_df, preds, labels = evaluate_model(model, tokenizer, subset_test_dataset, verbose = False, total = 500)

100%|██████████| 500/500 [04:48<00:00,  1.73it/s]


In [ ]:
zero_shot_accuracy, conf_matrix_zero_shot =  compute_stats(out_df, preds, labels)

overall accuracy: 
0.606
confusion matrix: 
correct forward 303 wrong forward 197 correct backward 284 wrong_backward 216


In [ ]:
import json

model_id_string = model_id.replace("/", "-")

# Saving DataFrame to CSV
out_df.to_csv(f'output_df_{model_id_string}.csv', sep="\t", index=False)

# Saving other data as JSON
data_to_save = {
    "preds": preds,
    "labels": labels,
    "zero_shot_accuracy": zero_shot_accuracy,
    "conf_matrix_zero_shot": conf_matrix_zero_shot
}

with open(f'output_data_{model_id_string}.json', 'w') as file:
    json.dump(data_to_save, file)

import pickle

data_to_save_pick = {
    "out_df": out_df,
    "preds": preds,
    "labels": labels,
    "zero_shot_accuracy": zero_shot_accuracy,
    "conf_matrix_zero_shot": conf_matrix_zero_shot
}

with open(f'pickle_output_data_{model_id_string}.pkl', 'wb') as file:
    pickle.dump(data_to_save_pick, file)



## Finetuning

In [ ]:
# from peft import prepare_model_for_kbit_training

# model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model)

In [ ]:
def print_modules(model, prefix=''):
    for name, module in model.named_children():
        full_name = f"{prefix}.{name}" if prefix else name
        print(full_name)
        print_modules(module, full_name)

print_modules(model)

layers
layers.0
layers.0.wte
layers.0.drop
layers.1
layers.1.ln
layers.1.resid_dropout
layers.1.mixer
layers.1.mixer.rotary_emb
layers.1.mixer.Wqkv
layers.1.mixer.out_proj
layers.1.mixer.inner_attn
layers.1.mixer.inner_attn.drop
layers.1.mixer.inner_cross_attn
layers.1.mixer.inner_cross_attn.drop
layers.1.mlp
layers.1.mlp.fc1
layers.1.mlp.fc2
layers.1.mlp.act
layers.2
layers.2.ln
layers.2.resid_dropout
layers.2.mixer
layers.2.mixer.rotary_emb
layers.2.mixer.Wqkv
layers.2.mixer.out_proj
layers.2.mixer.inner_attn
layers.2.mixer.inner_attn.drop
layers.2.mixer.inner_cross_attn
layers.2.mixer.inner_cross_attn.drop
layers.2.mlp
layers.2.mlp.fc1
layers.2.mlp.fc2
layers.2.mlp.act
layers.3
layers.3.ln
layers.3.resid_dropout
layers.3.mixer
layers.3.mixer.rotary_emb
layers.3.mixer.Wqkv
layers.3.mixer.out_proj
layers.3.mixer.inner_attn
layers.3.mixer.inner_attn.drop
layers.3.mixer.inner_cross_attn
layers.3.mixer.inner_cross_attn.drop
layers.3.mlp
layers.3.mlp.fc1
layers.3.mlp.fc2
layers.3.mlp.act


In [12]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [23]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    #target_modules=["query_key_value"],
    target_modules=["Wqkv"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model_peft = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 1572864 || all params: 815863808 || trainable%: 0.1927851173905731


Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

In [14]:
def map_concatenation_and_tokenization(samples):
    concatenated_phrases = []
    input_ids_list = []
    attention_mask_list = []

    for i in range(len(samples['startphrase'])):
        # Choose the ending based on the labels value for each sample in the batch
        ending = samples['ending1'][i] if samples['labels'][i] == 0 else samples['ending2'][i]
        concatenated_phrase = samples['startphrase'][i] + ' -> ' + ending
        concatenated_phrases.append(concatenated_phrase)

        # Tokenize the concatenated_phrase
        tokens = tokenizer(concatenated_phrase, truncation=True, max_length=512, return_tensors='pt')
        input_ids_list.append(tokens['input_ids'][0].tolist())
        attention_mask_list.append(tokens['attention_mask'][0].tolist())

    return {
        'concatenated_phrase': concatenated_phrases,
        'input_ids': input_ids_list,
        'attention_mask': attention_mask_list
    }

# Apply the mapping function
data = load_dataset("nightingal3/fig-qa")
data = data.map(map_concatenation_and_tokenization, batched=True)


Map:   0%|          | 0/9674 [00:00<?, ? examples/s]

Map:   0%|          | 0/1094 [00:00<?, ? examples/s]

Map:   0%|          | 0/1146 [00:00<?, ? examples/s]

Run the cell below to run the training! For the sake of the demo, we just ran it for few steps just to showcase how to use this integration with existing tools on the HF ecosystem.

In [ ]:
model_peft.to(device)

In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model_peft,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=20,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=1000,
        learning_rate=0.01, # 2e-4
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model_peft.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.


Step,Training Loss
1,4.600900
2,4.712600
3,3.637100
4,27.987700
5,8.601100
6,7.836300
7,7.371800
8,8.840500
9,7.084500
10,6.766200


`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it 

Step,Training Loss
1,4.600900
2,4.712600
3,3.637100
4,27.987700
5,8.601100
6,7.836300
7,7.371800
8,8.840500
9,7.084500
10,6.766200


`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it 

In [17]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [18]:
lora_config = LoraConfig.from_pretrained('outputs')
model_finetuned = get_peft_model(model_peft, lora_config)

In [19]:
text = """I'm selfish, impatient and a little insecure."""

device = "cuda:0"

model_finetuned.to(device)

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model_finetuned.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1462: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


I'm selfish, impatient and a little insecure. I don't want to wait for anyone."

"Well, that's not a very good attitude," said Sarah. "You should try to be more patient and understanding. It's not always easy, but it's worth it."

"I know, I know," said Emily. "But it's hard when you're in a hurry or when you're feeling stressed."

"I understand," said Sarah. "But maybe you could try to take a few deep breaths and calm down before you start. And maybe you could try to be more patient with yourself and with others."

"That's a good idea," said Emily. "I'll try to remember that."

As they continued to talk, they


In [20]:
out_df_finetuned, preds_finetuned, labels_finetuned = evaluate_model(model_finetuned, tokenizer, subset_test_dataset, verbose = False, total = 500)

100%|██████████| 500/500 [03:45<00:00,  2.22it/s]


In [21]:
zero_shot_accuracy_finetuned, conf_matrix_zero_shot_finetuned =  compute_stats(out_df_finetuned, preds_finetuned, labels_finetuned)

overall accuracy: 
0.608
confusion matrix: 
correct forward 304 wrong forward 196 correct backward 284 wrong_backward 216


In [ ]:
import json

model_id_string = model_id.replace("/", "-")

# Saving DataFrame to CSV
out_df_finetuned.to_csv(f'output_df_{model_id_string}_finetuned.csv', sep="\t", index=False)

# Saving other data as JSON
data_to_save = {
    "preds": preds_finetuned,
    "labels": labels_finetuned,
    "zero_shot_accuracy": zero_shot_accuracy_finetuned,
    "conf_matrix_zero_shot": conf_matrix_zero_shot_finetuned
}

with open(f'output_data_{model_id_string}_finetuned.json', 'w') as file:
    json.dump(data_to_save, file)

import pickle

data_to_save_pick = {
    "out_df": out_df_finetuned,
    "preds": preds_finetuned,
    "labels": labels_finetuned,
    "zero_shot_accuracy": zero_shot_accuracy_finetuned,
    "conf_matrix_zero_shot": conf_matrix_zero_shot_finetuned
}

with open(f'pickle_output_data_{model_id_string}_finetuned.pkl', 'wb') as file:
    pickle.dump(data_to_save_pick, file)

